# PEFT

`PEFT` (Parameter-Efficient Fine-Tuning) es una librería para adaptar de manera eficiente grandes modelos previamente entrenados a varias aplicaciones posteriores sin ajustar todos los parámetros de un modelo porque es prohibitivamente costoso.

Los métodos `PEFT` solo ajustan una pequeña cantidad de parámetros (adicionales) del modelo, lo que reduce significativamente los costos computacionales y de almacenamiento, al tiempo que producen un rendimiento comparable a un modelo completamente ajustado. Esto hace que sea más accesible entrenar y almacenar modelos de lenguajes grandes (LLM) en hardware de consumo.

`PEFT` está integrado con las bibliotecas [transformers](https://maximofn.com/hugging-face-transformers/), `difusers` y [accelerate](https://maximofn.com/hugging-face-accelerate/) para proporcionar una forma más rápida y sencilla de cargar, entrenar y utilizar modelos grandes para inferencia y entrenamiento

`PEFT` ofrece métodos eficientes en parámetros para ajustar grandes modelos previamente entrenados.

El paradigma tradicional es ajustar todos los parámetros de un modelo para cada tarea, pero esto se está volviendo extremadamente costoso y poco práctico debido a la enorme cantidad de parámetros en los modelos actuales.

En cambio, es más eficiente entrenar una cantidad menor de parámetros o utilizar un método de reparametrización como la adaptación de rango bajo ([LoRA - low rank adapter](https://maximofn.com/lora/)) para reducir la cantidad de parámetros entrenables.

## Instalación

Para instalar `PEFT` hay que ejecutar en la terminal

```bash
pip install peft
```

# Vistazo rápido del entrenamiento con PEFT

Cada método `PEFT` está definido por una clase `PeftConfig` que almacena todos los parámetros importantes para construir un `PeftModel`. Por ejemplo, para entrenar con `LoRA`, cargamos y creamos una clase `LoraConfig` y especificamos los siguientes parámetros:

 * `task_type`: la tarea para la que se va a entrenar (secuencia a secuencia en este caso)
 * `inference_mode`: si estamos usando el modelo para inferencia o no
 * `r`: la dimensión de las matrices de bajo rango (`rank`), en este caso 8
 * `lora_alpha`: el factor de escala para las matrices de rango bajo, en este caso 32
 * `lora_dropout`: la probabilidad de dropout de las capas LoRA, en este caso 0.1

In [1]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

Se necesita un modelo base, que puede cargar desde la biblioteca de transformers

In [1]:
from transformers import AutoModelForSeq2SeqLM

# model = AutoModelForSeq2SeqLM.from_pretrained("openai-community/gpt2")
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/mt0-large")

config.json:   0%|          | 0.00/800 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/repos/69/cd/69cd50da0d4ff75491317a19d706f489d842f63c0e2d070a45087b5df49c299b/4f9e1961a362116b17c9e33a458394baed48f9147de767952086d5c4c662c018?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1721646610&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMTY0NjYxMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82OS9jZC82OWNkNTBkYTBkNGZmNzU0OTEzMTdhMTlkNzA2ZjQ4OWQ4NDJmNjNjMGUyZDA3MGE0NTA4N2I1ZGY0OWMyOTliLzRmOWUxOTYxYTM2MjExNmIxN2M5ZTMzYTQ1ODM5NGJhZWQ0OGY5MTQ3ZGU3Njc5NTIwODZkNWM0YzY2MmMwMTg%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=EG-T5-3vkHkyEibcrLTZaWXP3sZ-mtDGMIyKg7R5iGs8AahUGUHNNhW5QDWEn-M9RWDyvmTcpy%7EorfHTiH8FuBzdcrLntVZv1KLiZ2bCSIMePRlDPmqmDJO%7EUGOThtlN0MNcjzeUSXOiK57z2Si9vksliarDFt2p4AoPXW%7E0He1-wuRWgvIb-NU-TyUqz9kVfLsPeXxBjj0MsOCF-RdFfc7updTgc-B%7EDBhxQcZS721MMM

model.safetensors:  93%|#########3| 4.58G/4.92G [00:00<?, ?B/s]

Una vez configurado `LoraConfig`, creamos un `PeftModel` con la función `get_peft_model()`. Se le pasa el modelo base y la configuración `LoraConfig` que contiene los parámetros sobre cómo configurar un modelo para entrenar con LoRA.

Envolvemos el modelo base y la configuración `lora_config` con la función `get_peft_model()` para crear un `PeftModel`. Para tener una idea de la cantidad de parámetros entrenables en el modelo, utilizamos el método `print_trainable_parameters`

In [3]:
from peft import get_peft_model

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 1,231,940,608 || trainable%: 0.1915


De los 1,2B de parámetros de `bigscience/mt0-large`, solo estamos entrenando el 0,19% de ellos, es decir 2,3M de parámetros.

Ya podemos entrenar el modelo, para ello primero descargamos un dataset

In [8]:
from datasets import load_dataset

dataset_repo = "Maximofn/opus100"
opus100_train = load_dataset(dataset_repo, split='train')
opus100_val = load_dataset(dataset_repo, split='validation')
opus100_test = load_dataset(dataset_repo, split='test')

In [15]:
opus100_train[0]['translation']

{'en': "It was the asbestos in here, that's what did it!",
 'es': 'Fueron los asbestos aquí. ¡Eso es lo que ocurrió!'}

Definimos un tokenizador

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Definimos una función de tokenización

In [50]:
def tokenize_function(examples):
    return {
        'en_ids': tokenizer(examples["translation"]['en'], padding=True, truncation=True, max_length=30),
        'es_ids': tokenizer(examples["translation"]['es'], padding=True, truncation=True, max_length=30),
    }

Tokenizamos el dataset

In [60]:
opus100_train_tokenized = opus100_train.map(tokenize_function)#, batched=True)
opus100_val_tokenized = opus100_val.map(tokenize_function)#, batched=True)
opus100_test_tokenized = opus100_test.map(tokenize_function)#, batched=True)

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Creamos una métrica de evaluación

In [61]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Creamos un trainer

In [62]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="Maximofn/bigscience/mt0-large-lora",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

Y ahora entrenamos

In [64]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=opus100_train_tokenized,
    eval_dataset=opus100_val_tokenized,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/wallabot/miniconda3/envs/nlp/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/31250 [00:00<?, ?it/s]

IndexError: Invalid key: 998078 is out of bounds for size 0

## Vistazo rápido de la inferencia

In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

In [8]:
model = AutoPeftModelForCausalLM.from_pretrained("ybelkada/opt-350m-lora")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

model = model.to("cuda")
model.eval()

inputs = tokenizer("Preheat the oven to 350 degrees and place the cookie dough", return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=50)
print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

Preheat the oven to 350 degrees and place the cookie dough in the center of the oven.

In a large bowl, combine the flour, baking powder, baking soda, salt, and cinnamon.

In a separate bowl, combine the egg yolks, sugar, and vanilla.


